# Growing degree days & Extreme degree days calculations

## Packages

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import dask
import os

## Functions

In [4]:
from functions_2a import degreeDays, yearlyCalculationSum

In [3]:
filein_base_yearly = "/storage/home/cta5244/work/pyWBM_yield_data/NCEPNARR_NLDAS_tmax_tmin"
start_year = 1979
end_year = 2025

In [4]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="1GiB",
    walltime="03:00:00",
)

cluster.scale(jobs=50) 

/storage/home/cta5244/mambaforge/envs/pyWBM/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34069 instead
  warnings.warn(
2025-03-18 13:57:23,789 - tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x1471f17164a0>>, <Task finished name='Task-18' coro=<SpecCluster._correct_state_internal() done, defined at /storage/home/cta5244/mambaforge/envs/pyWBM/lib/python3.10/site-packages/distributed/deploy/spec.py:346> exception=RuntimeError("can't start new thread")>)
Traceback (most recent call last):
  File "/storage/home/cta5244/mambaforge/envs/pyWBM/lib/python3.10/site-packages/tornado/ioloop.py", line 750, in _run_callback
    ret = callback()
  File "/storage/home/cta5244/mambaforge/envs/pyWBM/lib/python3.10/site-packages/tornado/ioloop.py", line 774

In [23]:
from dask.distributed import Client

client = Client(cluster)

In [24]:
client

<Client: 'tcp://146.186.150.11:43051' processes=50 threads=50, memory=50.00 GiB>

In [25]:
results = []
for year in np.arange(start_year, end_year, 1):
    out = dask.delayed(yearlyCalculationSum)(year=year, filein_base_yearly=filein_base_yearly)
    results.append(out)

In [26]:
results = dask.compute(*results)